In [11]:
PROJECT_NUMBER = '569083142710'
TRAINING_REPOSITORY = 'trainings'
TRAIN_IMAGE_URI = f'us-central1-docker.pkg.dev/jchavezar-demo/{TRAINING_REPOSITORY}/train_lightgbm_cpu:latest'
PIPELINE_ROOT_PATH = 'gs://vtx-root-path'

In [3]:
!rm -fr custom_5
!mkdir custom_5
!mkdir custom_5/trainer

In [22]:
%%writefile custom_5/trainer/train.py

import os

print(os.listdir())
print(os.listdir('/mnt'))

with open('/mnt/nfs/nfs_models/1.txt', 'w') as f:
    f.write('test')

Overwriting custom_5/trainer/train.py


In [23]:
%%writefile custom_5/Dockerfile

FROM python:3.7-buster

COPY trainer /trainer

ENTRYPOINT ["python", "trainer/train.py"]

Overwriting custom_5/Dockerfile


In [24]:
!gcloud builds submit -t $TRAIN_IMAGE_URI custom_5/.

Creating temporary tarball archive of 2 file(s) totalling 219 bytes before compression.
Uploading tarball of [custom_5/.] to [gs://jchavezar-demo_cloudbuild/source/1660309198.244855-d7fe7a5babf944458f76c4704f0aa853.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/jchavezar-demo/locations/global/builds/a20ddce3-8c86-4615-a5d7-6906216e0dac].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/a20ddce3-8c86-4615-a5d7-6906216e0dac?project=569083142710 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "a20ddce3-8c86-4615-a5d7-6906216e0dac"

FETCHSOURCE
Fetching storage object: gs://jchavezar-demo_cloudbuild/source/1660309198.244855-d7fe7a5babf944458f76c4704f0aa853.tgz#1660309198849201
Copying gs://jchavezar-demo_cloudbuild/source/1660309198.244855-d7fe7a5babf944458f76c4704f0aa853.tgz#1660309198849201...
/ [1 files][  339.0 B/  339.0 B]                                                
Operation completed over 

In [29]:
    ### Training Worker Pool specs

train_worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": "n1-standard-32",
            "accelerator_type": "NVIDIA_TESLA_T4",
            "accelerator_count": 4
        },
        "replica_count": "1",
        "nfs_mounts": [
            {
            "server": "10.69.176.2",
            "path": "/nfs_models",
            "mount_point": "nfs_models"
            }
        ],
        "container_spec": {
            "image_uri": TRAIN_IMAGE_URI,
            "command": [
                "python",
                "trainer/train.py"
            ],
        }
    },
]

In [30]:
from kfp.v2.dsl import pipeline
from google_cloud_pipeline_components.v1 import custom_job


@pipeline(name='lightgbm-pipe')
def pipeline():
    custom_job_task = custom_job.CustomTrainingJobOp(
        project='jchavezar-demo',
        display_name='test',
        worker_pool_specs=train_worker_pool_specs,
        network=f'projects/{PROJECT_NUMBER}/global/networks/default'
    )

### Compile Pipeline

In [31]:
from kfp.v2 import compiler
import warnings
warnings.filterwarnings('ignore')

compiler.Compiler().compile(pipeline_func=pipeline,
        package_path='lightgbm.json')

### Run Pipeline Job

In [32]:
import google.cloud.aiplatform as aip

job = aip.PipelineJob(
    display_name="lightgbm",
    template_path="lightgbm.json",
    pipeline_root=PIPELINE_ROOT_PATH,
    parameter_values={
    },
)

job.submit(service_account='vtx-pipe@jchavezar-demo.iam.gserviceaccount.com')

Creating PipelineJob
PipelineJob created. Resource name: projects/569083142710/locations/us-central1/pipelineJobs/lightgbm-pipe-20220812080644
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/569083142710/locations/us-central1/pipelineJobs/lightgbm-pipe-20220812080644')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/lightgbm-pipe-20220812080644?project=569083142710
